In [ ]:
from functions import *
import json

# Creating a P2PKH transaction

In this section we'll create a P2PKH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using the bitcoin core test framework.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](./0.2-hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](./0.4-addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](./0.3-bitcoin-script.ipynb)'

### Setup bitcoin core test framework
Start up regtest mode, delete any regtest network history so we are starting from scratch. 

In [ ]:
# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

Create a wallet so that we can fund our first output using the `sendtoaddress` command. Mine 101 blocks so that the first block subsidy will have sufficient block maturity (100 blocks) to spend from.

In [ ]:
# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, invalid_call=False)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

### Create a P2PKH UTXO

In order to create a transaction spending from a P2PKH UTXO, we'll first need to create the UTXO that is locked with a p2pkh script. To do that, we'll create a P2PKH address from a private key, and fund it using the bitcoind wallet created in the setup step.

#### Create a p2pkh address 
For more on this step, review the 'Addresses' notebook.

In [ ]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
sender_p2pkh_addr = pk_to_p2pkh(sender_pubkey, network = "regtest")
print("sender's p2pkh address: " + sender_p2pkh_addr)

#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [ ]:
txid_to_spend = node.sendtoaddress(sender_p2pkh_addr, 2.001)
print(txid_to_spend)

We can view the transaction using the bitcoin-cli commands `getrawtransaction` and `decoderawtransaction` as follows:

In [ ]:
raw_tx = node.getrawtransaction(txid_to_spend)
decoded = node.decoderawtransaction(raw_tx)
print(json.dumps(decoded, indent=2, default=str))

#### Find which output index the btc was sent to
Since we only sent 2.001 btc of the coinbase transaction (50 btc) to our address, bitcoind creates a change output to send the rest of the btc. By looking at the outputs we can see which is the change output and which was sent to our address. To do this in python we can do the following:

In [ ]:
if decoded["vout"][0]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 0
elif decoded["vout"][1]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 1
else:
    raise Exception("couldn't find output")
print("index to spend from: " + str(index_to_spend))

#### Mine a block so that the funding tx gets confirmed

In [ ]:
node.generatetoaddress(1, address, invalid_call=False);

## Spending a p2pkh UTXO

Now that we have some funds locked up in a p2pkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

### Decoding a base58 address

The first thing we need to do is decode the address. This lets us:  
1 - validate the checksum to know the address was transmitted without error  
2 - make sure we are sending btc on the correct network (testnet/mainnet)  
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the '[Addresses](./0.4-addresses.ipynb)' chapter.

In [ ]:
receiver_address = 'mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE'
receiver_address_decoded = decode_base58(receiver_address)
# TODO: create a function in the address chapter to validate and parse addresses and use here

prefix = receiver_address_decoded[0]  
pubkey_hash = receiver_address_decoded[1:-4] 
checksum = receiver_address_decoded[-4:]
print(hex(prefix))
print(pubkey_hash.hex())
print(checksum.hex())

The first byte , in our case `6f`, tells us that this address corresponds to a p2pkh output for testnet. For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [ ]:
receiver_spk = bytes.fromhex("76a914") + pubkey_hash + bytes.fromhex("88ac")

### Create an unsigned p2pkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [ ]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the input's scriptSig, the field where the signature goes, is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [ ]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

We can decode this raw transaction to inspect it and see that it has all the information we need apart from the scriptSig.

In [ ]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

Before we can sign this transaction there is one final step we need to do. We need to replace the empty scriptSig with the scriptPubkey of the input we are signing over. If we had multiple inputs, we would need to do this step for each input. We will cover signing transactions with multiple inputs in a later chapter.

Since we are spending from a p2pkh utxo, we will create the scriptPubkey in the same way as we did for the outputs, but using the sender's pubkey:

In [ ]:
pk_hash = hash160(sender_pubkey)
input_spk = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk # replace the empty scriptSig with the input scriptPubkey
    + sequence
)

# tx hex to sign
tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [ ]:
# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

print("signed transaction: ",signed_tx.hex())

### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [ ]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
# new_tx_txid = node.testmempoolaccept(signed_tx.hex())

print(new_tx_txid)

We can decode the serialized transaction using ```decoderawtransction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [ ]:
print("receiver's p2pkh address: " + receiver_address)
change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
print("sender's change p2pkh address: " + change_p2pkh_addr)

In [ ]:
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

## Quiz
1. How is the transaction ID calculated? 
2. Which field of a transaction contains the signature(s)? 
3. Bitcoin ECDSA signatures are vulnerable to third party malleation. As a result, it is possible for a third party to alter the transaction ID. If Alice is broadcasting a transaction to Bob, how might a third party manage to change the transaction ID using only publicly available information?
4. What problems or limitations does this transaction malleability cause?

 ## Answers
1. It is the double SHA256 (HASH256) of the raw transaction. Note that the output is displayed in little endian. See '[The First Bitcoin Transaction (Pay to Pubkey)](./1.1-first-btc-tx.ipynb)' for an example.
2. The scriptSig in the input field contains the signature(s).
3. A third party could observe the transaction in the mempool, change the signature, then broadcast the transaction with the altered signature and transaction ID.
4. Two issues (non exhaustive):
  - It makes it impossible to create a chain of offline transactions as the inputs of a child may change by the time it is confirmed. This is an issue for many layer 2 protocols such as lightning payment channels.
  - If Alice or Bob are searching for the transaction using the transaction ID, they may never find it. 

## Exercise
Now that we have the change output with a known private key `change_privkey`, create a transaction that spends from it to two outputs. The first output should to send 0.1 btc to the address `mgiS1dSDrPunE7GvXmoS4xEfmdwWsStZc7`. The second address should send the rest to a change output with the address `mz8AXDhDMhvLs7kxwfQxvcH5GoVH6AdARZ`. Set the miner fee to 0.001 btc.

The first step will be to decode the output addresses we want to send btc to and convert them into their scriptPubkeys.

## Solution

In [ ]:
# Solution - Converting the addresses to scriptPubKeys
receiver_address = 'mgiS1dSDrPunE7GvXmoS4xEfmdwWsStZc7'
receiver_address_decoded = decode_base58(receiver_address)
receiver_pubkey_hash = receiver_address_decoded[1:-4] 
output1_spk = bytes.fromhex("76a914") + receiver_pubkey_hash + bytes.fromhex("88ac")

change_address = 'mz8AXDhDMhvLs7kxwfQxvcH5GoVH6AdARZ'
change_address_decoded = decode_base58(change_address)
change_pubkey_hash = change_address_decoded[1:-4] 
output2_spk = bytes.fromhex("76a914") + change_pubkey_hash + bytes.fromhex("88ac")

Now let's create an unsigned transaction. For now we will keep the scriptSig empty (`0x00`)

In [ ]:
# Solution - creating an unsigned transaction
version = bytes.fromhex("0200 0000")

# INPUTS
input_count = bytes.fromhex("01")

# INPUT 1
txid_to_spend = new_tx_txid
txid = (bytes.fromhex(txid_to_spend))[::-1]
index_to_spend = 1
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)
scriptsig = bytes.fromhex("00")
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value_sat = int(float("0.1") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)

# OUTPUT 2
output2_value_sat = int(float("0.399") * 100000000) # 0.5 - 0.1 - 0.001 for the miner fee
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

# print(unsigned_tx.hex())

Replace the scriptSig with the input's scriptPubKey. You can either code this yourself or take it directly from the decoded transaction. The `"hex"` field of the output we are spending from can be used directly.

In [ ]:
# Solution - replace the scriptPubKey for creating the sigHash
input_spk = bytes.fromhex("76a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac")

In [ ]:
# Solution - creating the sigHash and signing the transaction
inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk
    + sequence
)

tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)


# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(change_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(change_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

# print("signed transaction: ",signed_tx.hex())

In [ ]:
# Try broadcasting the transaction to see if it works!
node.sendrawtransaction(signed_tx.hex())

In [ ]:
# stop bitcoin core
test.shutdown()